# boolean함수의 Demonstration과 ipywidgets

이번 챕터에서는 iPython Widget에서 사용된 logic tools overlay 안에 있는 boolean 함수를 검증할 것이다.This notebook demonstrates the boolean functions in the logic tools overlay used with iPython Widgets.


## 슬라이드쇼

이 notebook은 슬라이드쇼로 실행되기 위해 만들어졌으나, 일반 notebook으로도 실행될 수 있다.

View > Cell Toolbar > Slideshow로 가면 *Slideshow options*를 볼 수 있다. 거기서 프레젠테이션에 포함되거나 제외 될 슬라이드를 선택할 수 있다.

코드 셀을 클릭하고 Ctrl + Enter를 눌러 슬라이드 쇼보기에서 코드 셀을 실행할 수 있습니다.


## 데모를 실행하기 위한 지침:

* slideshow view로 들어가기 전에 Cell> Run All을 클릭하여 노트의 모든 코드를 실행하십시오.

데모를 작성하는 데 사용한 코드를 notebook과 예제에서 찾아 볼 수 있다.

슬라이드 쇼 모드에서 커서 키 (왼쪽)를 사용하여 프레젠테이션을 탐색한다.

* 슬라이드 쇼를 입력하고 보기를 숨기려면 Alt + r을 누르십시오. (동일한 키 Alt + r로 슬라이드 쇼 모드 종료)

In [15]:
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display

from pynq.overlays.logictools import LogicToolsOverlay
logictools_olay = LogicToolsOverlay('logictools.bit') # Download the logictools overlay

In [16]:
def update_boolean_generator(sender):
    led = output.value.replace("E", "") # Change "LED" to "LD"
    led = led.replace("NOT ", "~") # Replace the text "NOT" with the "~" (NOT operator in Python)
    
    logic = []
    for i in [logic0.value, logic1.value, logic2.value]:
        if i is 'AND' :
            logic.append('&')
        elif i is 'OR' :
            logic.append('|')
        elif i is 'XOR' :
            logic.append('^')
    
    inputs = []
    for i in [input0.value, input1.value, input2.value, input3.value]:
        if i is not '-' :
            i = i.replace("NOT ", "~") 
            inputs.append(i)
            
    # Build logic function
    # Assumes inputs[0] always exists
    function = led + '=' + inputs[0]

    for i, j in zip(logic, inputs[1:4]):
        function = function + i + j

    # Apply the function to the boolean generator    
    boolean_generator = logictools_olay.boolean_generator
    boolean_generator.reset()
    boolean_generator.setup([function])
    boolean_generator.run()

# 논리 함수 위젯

 선택된 위젯으로 구성된 간단한 GUI가 만들어 져서 각 LED에 대해 논리 기능을 만들 수 있습니다. 각 논리 함수는 4 개의 push버튼들과 사용 가능한 Boolean operator들로 만들어질 수 있다.

LED를 선택하여 *Output*을 만든다.

In [17]:
output = widgets.Select(
    options=['LED0', 'LED1', 'LED2', 'LED3'],
    value='LED0',
    description='Select LED',
    disabled=False)
output.layout.width='400px'

Input은 4개의 어떤 push botton이든지 가능하다.
Not 논리 연산자는 별도의 논리 연산자가 아닌 입력으로 캡슐화된다.

In [18]:
inputs = ['PB0', 'PB1', 'PB2', 'PB3', 'NOT PB0', 'NOT PB1', 'NOT PB2', 'NOT PB3', '-']
input0 = widgets.Select(
    options=inputs,
    value='PB0',
    #description=' = ',
    disabled=False)

input1 = widgets.Select(
    options=inputs,
    value='PB1',
    #description='PB',
    disabled=False)

input2 = widgets.Select(
    options=inputs,
    value='-',
    #description='PB',
    disabled=False)

input3 = widgets.Select(
    options=inputs,
    value='-',
    #description='PB',
    disabled=False)

논리연산자로는 *AND*, *OR*, *XOR*를 선택할수 있다.

In [19]:
logic_operators=['AND', 'OR', 'XOR', '-']

logic0 = widgets.Select(
    options=logic_operators,
    value='AND',
    disabled=False)

logic1 = widgets.Select(
    options=logic_operators,
    value='-',
    disabled=False)

logic2 = widgets.Select(
    options=logic_operators,
    value='-',
    disabled=False)

**입력값**이 바뀌면 Boolean 생성기를 업데이트 하십시오.

In [20]:
output.observe(update_boolean_generator, names='value')
logic0.observe(update_boolean_generator, names='value')
logic1.observe(update_boolean_generator, names='value')
logic2.observe(update_boolean_generator, names='value')
input0.observe(update_boolean_generator, names='value')
input1.observe(update_boolean_generator, names='value')
input2.observe(update_boolean_generator, names='value')
input3.observe(update_boolean_generator, names='value')

모든 item들을 Box 안에 그룹화시키고, update_boolean_generator를 호출하여 Boolean생성기를 초기화한다.

In [21]:
items = [output, input0, logic0, input1, logic1, input2, logic2, input3]
update_boolean_generator(_)

select_led = widgets.Box(items)

### Boolean 함수 만들기

Input은 **PB0**, **PB1**, **PB2**, **PB3** (PB의 반전인 **NOT PB0**, **NOT PB1** ...) 으로 네개의 push버튼으로 받을 수 있다.

Output은 **LD0**, **LD1**, **LD2**, **LD3** 로 총 네 개의 LED로 받을 수 있다.

Operation은 아래처럼 쓸 수 있다.
* **&** : AND
* **|** : OR
* **^** : XOR 

Push버튼과 LED를 사용하여 구현될 논리 함수를 선택하기 위해 Select Box를 사용해라.

In [22]:
select_led

Box(children=(Select(description='Select LED', layout=Layout(width='400px'), options=('LED0', 'LED1', 'LED2', …

보드위 push버튼을 눌러 논리함수를 테스트하고, LED 불이 제대로 켜지는지 확인하라.